In [1]:
from fastfusion import arch
from fastfusion import constraints
from fastfusion import workload
import fastfusion

variables = {"tech_node": 1e-9}
global_cycle_period = 1e-9

main_memory = arch.Memory(
    name="MainMemory",
    component_class="DRAM",
    attributes=dict(
        _size=4*1024*1024*1024*8,
        _bandwidth_reads_plus_writes_per_cycle=614e9 / 8192 * global_cycle_period,
        _datawidth=8,
        width=8192,
        depth=4*1024*1024*1024*8 / 8192,
    ))
main_memory.constraints.tensors.keep = "~Intermediates()"

global_buffer = arch.Memory(
    name="GlobalBuffer",
    component_class="SRAM",
    attributes=dict(
        _size=1024*1024*128*8,
        _datawidth=8,
        _bandwidth_reads_plus_writes_per_cycle=1,
        width=16384,
        depth=1024*1024*128*8 / 16384,
    )
)
global_buffer.constraints.tensors.keep = "All()"
global_buffer.constraints.tensors.no_refetch_from_above = "All() if ~MainMemory.tensors() else Nothing()"
global_buffer.constraints.dataflow.tensor_order_options = [["MainMemory.tensors()", "~MainMemory.tensors()"]]

local_buffer = arch.Memory(
    name="LocalBuffer",
    component_class="SRAM",
    attributes=dict(
        _size=1024*1024*4*8,
        _datawidth=8,
        _bandwidth_reads_plus_writes_per_cycle=1,
        width=16384,
        depth=1024*1024*4*8 / 16384,
    ),
    spatial=[{"name": "Z", "fanout": 4}],
)
local_buffer.constraints.tensors.keep = "input | output"

local_buffer.constraints.dataflow.tensor_order_options = [["output", "input"]]
local_buffer.constraints.spatial.append(constraints.Spatial(
    name="Z",
    min_utilization="1 if len(All()) > 2 else 0",
    loop_bounds=[{"expression": "weight.rank_variables()", "operator": "==", "value": 1}]
))

register = arch.Memory(
    name="Register",
    component_class="dummy_storage",
    spatial=[{"name": "X", "fanout": 128}, {"name": "Y", "fanout": 128}],
    attributes=dict(
        _size=8,
        _datawidth=8,
    )
)
register.constraints.tensors.keep = "weight"
register.constraints.tensors.bypass = "~weight"
register.constraints.spatial.append(constraints.Spatial(
    name="X",
    min_utilization=1,
    loop_bounds=[{"expression": "input.rank_variables()", "operator": "==", "value": 1}]
))
register.constraints.spatial.append(constraints.Spatial(
    name="Y",
    min_utilization=1,
    loop_bounds=[{"expression": "output.rank_variables()", "operator": "==", "value": 1}]
))

compute = arch.Compute(
    name="MAC",
    component_class="intmac",
    attributes=dict(
        multiplier_width=8,
        adder_width=16,
    )
)
compute.constraints.misc.enabled = "len(All()) == 3"

scalar_unit = arch.Compute(
    name="scalar_unit",
    component_class="dummy_compute",
    attributes=dict(
        area=1,
        _computes_per_cycle=128
    )
)
scalar_unit.constraints.misc.enabled = "len(All()) == 2"


arch = arch.Arch(
    nodes=[
        main_memory,
        global_buffer,
        local_buffer,
        scalar_unit,
        register,
        compute,
    ],
    global_cycle_period=global_cycle_period,
)




acc0 = workload.TensorAccess(name="T0", projection=["m", "n0"])
acc1 = workload.TensorAccess(name="W0", projection=["n0", "n1"])
acc2 = workload.TensorAccess(name="T1", projection=["m", "n1"], output=True)
renames = {"input": "T0", "weight": "W0", "output": "T1"}
matmul1 = workload.Einsum(name="Matmul1", tensor_accesses=[acc0, acc1, acc2], renames=renames)

workload = workload.Workload(
    einsums=[matmul1],
    shape=dict(
        m="0 <= m < 128",
        n0="0 <= n0 < 64",
        n1="0 <= n1 < 128",
    ),
)

spec = fastfusion.Specification(
    arch=arch,
    workload=workload,
    variables=variables,
)
pmappings = fastfusion.mapper.FFM.make_pmappings(spec)

WARNING Loading configuration file from /root/.config/fastfusion/config.yaml
Generating pmapping templates for compute scalar_unit Einsum Matmul1: 0it [00:00, ?it/s]
Generating pmapping templates for compute MAC Einsum Matmul1: 18it [00:00, 63.39it/s]
Generating jobs: 100%|██████████| 2/2 [00:01<00:00,  1.57it/s]
WARNING Einsum Matmul1 has 18 pmapping templates:
WARNING 	[W0,T1,T0 in MainMemory] T-m,n0 [T0 in GlobalBuffer] T-n1 [T1 in GlobalBuffer] T-n0 [W0 in GlobalBuffer] S-Z-m T-m [T1 in LocalBuffer] T-n0 [T0 in LocalBuffer] S-Y-n0 S-X-n1 T-n1 [W0 in Register] T-m MAC computes Matmul1
WARNING 	[W0,T1,T0 in MainMemory] T-m,n0 [T0 in GlobalBuffer] T-n1 [W0 in GlobalBuffer] T-m [T1 in GlobalBuffer] S-Z-m [T1 in LocalBuffer] T-n0 [T0 in LocalBuffer] S-Y-n0 S-X-n1 T-n1 [W0 in Register] T-m MAC computes Matmul1
WARNING 	[W0,T1,T0 in MainMemory] T-m,n1 [T1 in GlobalBuffer] T-n0 [T0 in GlobalBuffer] T-n1 [W0 in GlobalBuffer] S-Z-m T-m [T1 in LocalBuffer] T-n0 [T0 in LocalBuffer] S-Y-n0 S-X-